In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())
print(params[0])

10
torch.Size([6, 1, 5, 5])
Parameter containing:
tensor([[[[-0.1990, -0.0501, -0.0519,  0.1296, -0.1993],
          [-0.1185, -0.0650,  0.1642,  0.0920,  0.0046],
          [-0.0480, -0.0404,  0.1146, -0.1208,  0.0898],
          [-0.1487, -0.1947, -0.0434, -0.0970, -0.1578],
          [ 0.1270,  0.1427,  0.0249,  0.0553,  0.0308]]],


        [[[-0.1845, -0.1856,  0.1506, -0.0989,  0.0833],
          [-0.1840,  0.0796,  0.1696,  0.0652,  0.0163],
          [-0.1341,  0.0821, -0.0473, -0.0122,  0.1098],
          [ 0.1986, -0.0941,  0.0654, -0.1417, -0.0639],
          [-0.1702,  0.1656, -0.1254, -0.0401, -0.1232]]],


        [[[ 0.0936, -0.0712,  0.0033,  0.1074, -0.1395],
          [ 0.0003,  0.0699, -0.1870, -0.0124, -0.1843],
          [-0.0655, -0.1193, -0.1535,  0.0418, -0.0075],
          [ 0.0075,  0.0234,  0.0070,  0.0874, -0.0289],
          [ 0.0279,  0.0906,  0.0297, -0.0845,  0.0863]]],


        [[[ 0.1346, -0.0540,  0.0397,  0.1691,  0.0768],
          [-0.1918,  0.096

In [4]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0156,  0.0956,  0.0533,  0.0258,  0.0879,  0.0299,  0.0993,  0.0846,
         -0.0049, -0.0346]], grad_fn=<AddmmBackward>)


In [5]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [7]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output,target)
print(loss)

tensor(1.2973, grad_fn=<MseLossBackward>)


In [11]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions)
print(loss.grad_fn.next_functions[0][0].next_functions)

((<AddmmBackward object at 0x7f289d28fad0>, 0L),)
((<AccumulateGrad object at 0x7f289d28fa90>, 0L), (<ReluBackward0 object at 0x7f289d28fb10>, 0L), (<TBackward object at 0x7f289d28fb90>, 0L))


In [12]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0112,  0.0045, -0.0041, -0.0142, -0.0061, -0.0257])


In [13]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [14]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output,target)
loss.backward()   ## Get the weights gradient
optimizer.step()  ## Dose the weights update